In [1]:
from __future__ import print_function

#data handling/prediction
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.externals.six import StringIO
import os

#visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
import pydotplus
from sklearn.tree import export_graphviz

#set variable to actually use graphviz
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'


In [2]:
#import data
trainData  = pd.read_csv("../data/train.csv")
#testData = pd.read_csv("../data/test.csv")
#can't use the provided test data at this point 
#because don't have survival values to calculate RMSE
trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
colList = trainData.columns.tolist()
colList

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [4]:
colList = ['Name','Ticket','Cabin','PassengerId', 'Survived', 'Age','Sex', 'Embarked', 'Pclass','SibSp','Parch','Fare']

In [5]:
working = trainData[colList]
working.head()

,Name,Ticket,Cabin,PassengerId,Survived,Age,Sex,Embarked,Pclass,SibSp,Parch,Fare
0,"Braund, Mr. Owen Harris",A/5 21171,NaN,1,0,22.0,male,S,3,1,0,7.2500
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",PC 17599,C85,2,1,38.0,female,C,1,1,0,71.2833
2,"Heikkinen, Miss. Laina",STON/O2. 3101282,NaN,3,1,26.0,female,S,3,0,0,7.9250
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",113803,C123,4,1,35.0,female,S,1,1,0,53.1000
4,"Allen, Mr. William Henry",373450,NaN,5,0,35.0,male,S,3,0,0,8.0500


In [6]:
working.shape

(891, 12)

In [7]:
trainData.shape

(891, 12)

In [8]:
#change gender to number
clean_sex = {"Sex": {"male": 1, "female": 0}}
clean_sex


{'Sex': {'female': 0, 'male': 1}}

In [9]:
working.replace(clean_sex, inplace=True)

In [10]:
#confirm casting was done right
working.Sex.value_counts()

1    577
0    314
Name: Sex, dtype: int64

In [11]:
working.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [12]:
cleaned = {"Embarked": {"S": 1, "C": 2, "Q": 3}}
working.replace(cleaned, inplace=True)
working.Embarked.value_counts()

1.0    644
2.0    168
3.0     77
Name: Embarked, dtype: int64

In [13]:
#fill in blank Embarked values with 1.0 as vast majority of passengers boarded at Southampton.
working = working.fillna({"Embarked": 1})
working.Embarked.value_counts()

1.0    646
2.0    168
3.0     77
Name: Embarked, dtype: int64

In [14]:
working.columns.tolist()

['Name',
 'Ticket',
 'Cabin',
 'PassengerId',
 'Survived',
 'Age',
 'Sex',
 'Embarked',
 'Pclass',
 'SibSp',
 'Parch',
 'Fare']

In [15]:
#exclude Age in col 5 for now - the nulls are throwing error when try to calculate RMSE
X, y = working.iloc[:, 6:], working.iloc[:, 4]

In [17]:
#for development purposes, use test_train_split on the cleaned train data so I have the survival value....
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.4, random_state=1234)

In [18]:
#define features
feature_names_tr = X_train.columns.tolist()
feature_names_te = X_test.columns.tolist()
print("X_train shape:", X_train.shape)
print("feature_names_tr:", feature_names_tr)
print("X_test shape:", X_test.shape)
print("feature_names_te:", feature_names_te)

X_train shape: (534, 6)
feature_names_tr: ['Sex', 'Embarked', 'Pclass', 'SibSp', 'Parch', 'Fare']
X_test shape: (357, 6)
feature_names_te: ['Sex', 'Embarked', 'Pclass', 'SibSp', 'Parch', 'Fare']


In [18]:
#Random forest of 500 trees
rf = RandomForestRegressor(n_estimators=500, bootstrap=True, oob_score=True, random_state=123)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest RMSE, unscaled & ignoring Age:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))


Random Forest RMSE, unscaled & ignoring Age: 0.405318299836


# Now with scaling....

In [19]:
#let's try that with scaling, since Fare can be much larger than 1. 
sc = StandardScaler()
X_scaled = X.copy()
X_scaled[feature_names_tr] = sc.fit_transform(X[feature_names_tr])

In [20]:
X_scaled.head()

,Sex,Embarked,Pclass,SibSp,Parch,Fare
0,0.737695,-0.568837,0.827377,0.432793,-0.473674,-0.502445
1,-1.355574,1.005181,-1.566107,0.432793,-0.473674,0.786845
2,-1.355574,-0.568837,0.827377,-0.474545,-0.473674,-0.488854
3,-1.355574,-0.568837,-1.566107,0.432793,-0.473674,0.420730
4,0.737695,-0.568837,0.827377,-0.474545,-0.473674,-0.486337


In [21]:
X_sc_train, X_sc_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=.4, random_state=1234)
rf = RandomForestRegressor(n_estimators=500, bootstrap=True, oob_score=True, random_state=123)
rf.fit(X_sc_train, y_train)
y_pred_rf = rf.predict(X_sc_test)

print("Random Forest RMSE, scaled, ignoring Age:", np.sqrt(mean_squared_error(y_test, y_pred_rf)))


Random Forest RMSE, scaled, ignoring Age: 0.405055372369


so scaling didn't make much difference. However, RMSE significantly worse (0.48 vs 0.40) when maintain single column for embarked vs. splitting into single column for each port as in previous attempt. 

# dealing with missing ages

In [19]:
#how often is Age 'NaN'?
working_noAge = working[working.Age.isnull()]
working_noAge.shape

(177, 12)

In [20]:
working_age = working[working.Age.notnull()]
working_age.describe()

,PassengerId,Survived,Age,Sex,Embarked,Pclass,SibSp,Parch,Fare
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000
mean,448.582633,0.406162,29.699118,0.634454,1.260504,2.236695,0.512605,0.431373,34.694514
std,259.119524,0.491460,14.526497,0.481921,0.521012,0.838250,0.929783,0.853289,52.918930
min,1.000000,0.000000,0.420000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,222.250000,0.000000,20.125000,0.000000,1.000000,1.000000,0.000000,0.000000,8.050000
50%,445.000000,0.000000,28.000000,1.000000,1.000000,2.000000,0.000000,0.000000,15.741700
75%,677.750000,1.000000,38.000000,1.000000,1.000000,3.000000,1.000000,1.000000,33.375000
max,891.000000,1.000000,80.000000,1.000000,3.000000,3.000000,5.000000,6.000000,512.329200


In [21]:
filter_F_noAge = np.logical_and(working.Age.isnull(), working.Sex==0)
filter_M_noAge = np.logical_and(working.Age.isnull(), working.Sex==1)
filter_F_Age = np.logical_and(working.Age.notnull(), working.Sex==0)
filter_M_Age = np.logical_and(working.Age.notnull(), working.Sex==1)

working_noAgeF = working[filter_F_noAge]
working_noAgeM = working[filter_M_noAge]
working_AgeF = working[filter_F_Age]
working_AgeM = working[filter_M_Age]
print("Mean age for F: ", working_AgeF.Age.mean())
print("Mean age for M: ", working_AgeM.Age.mean())

Mean age for F:  27.9157088123
Mean age for M:  30.7266445916


In [22]:
working[filter_F_noAge] = working[filter_F_noAge].fillna({"Age": working_AgeF.Age.mean()})

In [23]:
working[filter_M_noAge] = working[filter_M_noAge].fillna({"Age": working_AgeM.Age.mean()})
working.Age.describe()

count    891.000000
mean      29.736034
std       13.014897
min        0.420000
25%       22.000000
50%       30.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

# now try random forest again, including age this time

In [24]:
#exclude Age in col 5 for now - the nulls are throwing error when try to calculate RMSE
X_age, y_age = working.iloc[:, 5:], working.iloc[:, 4]

sc = StandardScaler()
X_scaled2 = X_age.copy()
X_scaled2[feature_names_tr] = sc.fit_transform(X_age[feature_names_tr])

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X_age, y_age, test_size=.4, random_state=1234)

#define features
feature_names_tr2 = X2_train.columns.tolist()
feature_names_te2 = X2_test.columns.tolist()
print("X_train shape:", X2_train.shape)
print("feature_names_tr:", feature_names_tr2)
print("X_test shape:", X2_test.shape)
print("feature_names_te:", feature_names_te2)

#Random forest of 500 trees
rf = RandomForestRegressor(n_estimators=500, bootstrap=True, oob_score=True, random_state=123)
rf.fit(X2_train, y2_train)
y2_pred_rf = rf.predict(X2_test)

print("Random Forest RMSE, scaled with Age:", np.sqrt(mean_squared_error(y2_test, y2_pred_rf)))


X_train shape: (534, 7)
feature_names_tr: ['Age', 'Sex', 'Embarked', 'Pclass', 'SibSp', 'Parch', 'Fare']
X_test shape: (357, 7)
feature_names_te: ['Age', 'Sex', 'Embarked', 'Pclass', 'SibSp', 'Parch', 'Fare']
Random Forest RMSE, scaled with Age: 0.390338799119


So the model is now a little better. 

Random Forest RMSE, unscaled & ignoring Age: 0.405318299836
Random Forest RMSE, scaled, ignoring Age: 0.405055372369
Random Forest RMSE, scaled with Age: 0.390338799119

...but what is actually being predicted? 


In [25]:
y2_pred_rf

array([ 1.        ,  0.00869149,  0.02133333,  0.998     ,  0.27      ,
        0.068     ,  0.88806667,  0.136     ,  0.32683333,  0.582     ,
        0.94986017,  0.084     ,  0.224     ,  0.004     ,  0.594     ,
        0.24186667,  0.        ,  0.3865599 ,  0.0665    ,  0.3252    ,
        0.01      ,  0.26      ,  1.        ,  0.026     ,  0.97      ,
        0.008     ,  0.004     ,  1.        ,  0.96795714,  0.032     ,
        0.47261592,  0.184     ,  0.51      ,  1.        ,  0.6       ,
        0.188     ,  0.134     ,  0.00933333,  0.606     ,  0.76      ,
        0.20614502,  0.958     ,  1.        ,  0.002     ,  0.011     ,
        0.226     ,  0.        ,  0.002     ,  0.286     ,  0.974     ,
        0.996     ,  0.034     ,  0.658     ,  0.002     ,  0.878     ,
        0.62723333,  0.092     ,  0.48233333,  0.0035    ,  0.042     ,
        0.422     ,  0.91      ,  0.996     ,  1.        ,  0.0045    ,
        1.        ,  0.79940786,  0.068     ,  0.        ,  0.17

...so the model isn't really predicting 0 or 1.  If force predictions to 0 or 1...

In [26]:
y2_pred_rf_bin = [0 if x <0.5 else 1 for x in y2_pred_rf]
y2_pred_rf_bin

[1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,


In [27]:
print("Random Forest RMSE, scaled with Age & response binned:", np.sqrt(mean_squared_error(y2_test, y2_pred_rf_bin)))


Random Forest RMSE, scaled with Age & response binned: 0.445959136941


# Lasso

In [28]:
#generate the polynomial transformer
pf_3 = PolynomialFeatures(degree=3, interaction_only=True)
#apply it to the data, but ignore the first constant column
pf_3_data = pf_3.fit_transform(X)
print(pf_3_data.shape)

lasso = Lasso()  # In practice be cognizant of convergence errors

mean_squared_errors_poly3_lasso = np.abs(
    cross_val_score(lasso, pf_3_data, y, cv=3))

rmses_lasso_poly3 = list(map(np.sqrt, mean_squared_errors_poly3_lasso))

print("3-fold mean RMSE for degree-3 case, strongest lasso regularization: ",
      np.mean(rmses_lasso_poly3))

(891L, 42L)
3-fold mean RMSE for degree-3 case, strongest lasso regularization:  0.382820411028


# Look at PCA
Initially said I was interested in which attributes determine who survived.  I have a model, but not which attributes are actually contributing the most.  

In [29]:
pca = PCA()
transformed_pca_x = pca.fit_transform(X2_train[feature_names_tr2])
#create component indices
component_names = ["component_"+str(comp) for comp in range(1, len(pca.explained_variance_)+1)]

#generate new component dataframe
transformed_pca_x = pd.DataFrame(transformed_pca_x,columns=component_names)
print(X2_train[feature_names_tr2].head())
transformed_pca_x.head()

       Age  Sex  Embarked  Pclass  SibSp  Parch     Fare
534  30.00    0       1.0       3      0      0   8.6625
755   0.67    1       1.0       2      1      1  14.5000
762  20.00    1       2.0       3      0      0   7.2292
605  36.00    1       1.0       3      1      0  15.5500
736  48.00    0       1.0       3      1      3  34.3750


,component_1,component_2,component_3,component_4,component_5,component_6,component_7
0,-23.069129,1.440149,-0.336933,0.148705,0.164799,0.443170,-0.840627
1,-18.286412,-28.081389,-0.111022,-0.226218,-0.998367,-0.276593,0.525780
2,-24.866500,-8.489508,-0.802451,0.211419,0.632509,-0.147635,0.358235
3,-15.966877,7.162587,0.593053,-0.425928,0.502970,0.634780,0.109753
4,3.294417,18.418820,2.341996,2.108375,-0.518210,0.611096,-0.284617


In [30]:
#generate component loadings on original features
component_matrix = pd.DataFrame(pca.components_,index=component_names,columns = feature_names_tr2)
